In [110]:
import numpy as np
import pandas as pd
import math

In [216]:
student_counts = pd.read_csv("data/student_counts.csv")
pair_counts = pd.read_csv("data/module_pairs.csv")
assignments = pd.read_csv("data/Assignments v2.csv")

In [ ]:
# Stress Score calculation = [weight(level)*1] + [weight(major)*1] + [weight(i_or_g)*2] + [weight(assignment_type)*3] + [weight(gap)*3] + weightage/100
# then standardize it to range 0 to 100

In [ ]:
# how important/stressful each of these factors are to students
# sum to 10?
# level: 1
# major: 1
# i_or_g: 2
# assignment_type: 3
# gap: 3

In [217]:
# how stressful each level within the factors are
weights = {
    "level": {
        "level_1k": 0.1,
        "level_2k": 0.2,
        "level_3k": 0.3,
        "level_4k": 0.4
    },
    "major": {
        "DSA": 0.4,
        "ST": 0.6
    },
    "i_or_g": {
        "I": 0.3,
        "G": 0.4,
        "I&G": 0.3
    },
    "assignment_type": {
        "Presentation": 0.1,
        "Project": 0.25,
        "Participation": 0.05,
        "Quiz": 0.1,
        "Assignment": 0.15,
        "Exam": 0.35
    },
    "gap": {
        "One Week": 0.4,
        "Two Weeks": 0.3,
        "More Than Two Weeks": 0.15,
        "Others": 0.15 # assume weight to be 0.2 if gap is not known eg. start date is NA
    }
}

In [218]:
assignments["Start Date"] = pd.to_datetime(assignments["Start Date"])
assignments["Due Date"] = pd.to_datetime(assignments["Due Date"])
assignments.drop(["Deadline Time"], axis=1, inplace=True)
assignments = assignments.sort_values(by=["Due Date"], na_position="last")

In [219]:
assignments.head()

,Module Code,Semester,Assignment Name,Weightage,Assignment Type,Group or Individual,Start Date,Due Date,Level,Major
4,DSA2101,2020,Quiz 1,NaN,Quiz,I,2021-01-12,2021-01-19,level_2k,DSA
5,DSA2101,2020,Assignment 1,2.5,Assignment,I,2021-01-19,2021-01-26,level_2k,DSA
6,DSA2101,2020,Assignment 2,2.5,Assignment,I,2021-02-09,2021-02-16,level_2k,DSA
9,DSA2101,2020,Take-Home Midterms,30.0,Assignment,I,2021-03-02,2021-03-06,level_2k,DSA
7,DSA2101,2020,Assignment 3,2.5,Assignment,I,NaT,2021-03-23,level_2k,DSA


In [150]:
#by_date = assignments.groupby("Due Date").agg(max)
#by_date

,Module Code,Semester,Assignment Name,Weightage,Assignment Type,Group or Individual,Start Date,Level,Major
Due Date,,,,,,,,,
2021-01-19,DSA2101,2020,Quiz 1,NaN,Quiz,I,2021-01-12,level_2k,DSA
2021-01-26,DSA2101,2020,Assignment 1,2.5,Assignment,I,2021-01-19,level_2k,DSA
2021-02-16,DSA2101,2020,Assignment 2,2.5,Assignment,I,2021-02-09,level_2k,DSA
2021-03-06,DSA2101,2020,Take-Home Midterms,30.0,Assignment,I,2021-03-02,level_2k,DSA
2021-03-23,DSA2101,2020,Assignment 3,2.5,Assignment,I,NaT,level_2k,DSA
...,...,...,...,...,...,...,...,...,...
2023-04-07,DSA3101,2220,Video Presentation,10.0,Presentation,G,NaT,level_3k,DSA
2023-04-10,ST4248,2220,Final Report,20.0,Project,G,NaT,level_4k,ST
2023-04-14,ST4248,2220,Term Paper,40.0,Project,I,2023-04-11,level_4k,ST


In [153]:
#by_date = by_date.reset_index()
#by_date

,Due Date,Module Code,Semester,Assignment Name,Weightage,Assignment Type,Group or Individual,Start Date,Level,Major
0,2021-01-19,DSA2101,2020,Quiz 1,NaN,Quiz,I,2021-01-12,level_2k,DSA
1,2021-01-26,DSA2101,2020,Assignment 1,2.5,Assignment,I,2021-01-19,level_2k,DSA
2,2021-02-16,DSA2101,2020,Assignment 2,2.5,Assignment,I,2021-02-09,level_2k,DSA
3,2021-03-06,DSA2101,2020,Take-Home Midterms,30.0,Assignment,I,2021-03-02,level_2k,DSA
4,2021-03-23,DSA2101,2020,Assignment 3,2.5,Assignment,I,NaT,level_2k,DSA
...,...,...,...,...,...,...,...,...,...,...
65,2023-04-07,DSA3101,2220,Video Presentation,10.0,Presentation,G,NaT,level_3k,DSA
66,2023-04-10,ST4248,2220,Final Report,20.0,Project,G,NaT,level_4k,ST
67,2023-04-14,ST4248,2220,Term Paper,40.0,Project,I,2023-04-11,level_4k,ST
68,2023-05-02,ST3247,2220,Finals,60.0,Exam,I,2023-05-02,level_3k,ST


In [220]:
def get_gap(due_date, start_date):
    if pd.isnull(start_date):
        return "Others"
    else:
        gap = (due_date - start_date).days
        if gap <= 7:
            return "One Week"
        elif gap <= 14:
            return "Two Weeks"
        else:
            return "More Than Two Weeks"

# Stress score per assignment

In [221]:
min_weightage = by_date["Weightage"].min()
max_weightage = by_date["Weightage"].max()
min_stress = min_weightage/100 * 0.1 * 3 + 0.3 * 2 + 0.1 * 1 + 0.4 * 1 + 0.15 * 3
max_stress = max_weightage/100* 0.35 * 3 + 0.4 * 2 + 0.4 * 1 + 0.6 * 1 + 0.4 * 3

In [222]:
stress = []

for i in range(0, len(by_date)):
        weightage = by_date.loc[i,"Weightage"]
        assignment_type = by_date.loc[i,"Assignment Type"]
        i_g = by_date.loc[i,"Group or Individual"]
        level = by_date.loc[i,"Level"]
        major = by_date.loc[i,"Major"]
        gap = get_gap(by_date.loc[i,"Due Date"], by_date.loc[i,"Start Date"])
        stress_score = weightage/100*weights['assignment_type'][assignment_type] * 3 + weights['i_or_g'][i_g] * 2 + weights['level'][level] * 1 + weights['major'][major] * 1 + weights['gap'][gap] * 3
        normalized_score = (stress_score - min_stress)/(max_stress - min_stress)*100
        stress.append(normalized_score)

In [224]:
stress

[nan,
 39.249655805415344,
 39.249655805415344,
 44.928866452501154,
 4.830197338228557,
 4.830197338228557,
 58.00826067003214,
 81.64295548416705,
 4.726938962826994,
 4.726938962826994,
 4.726938962826994,
 5.139972464433231,
 50.780174391922905,
 4.726938962826994,
 48.37081229921984,
 5.139972464433231,
 4.726938962826994,
 4.726938962826994,
 4.726938962826994,
 4.726938962826994,
 6.37907296925196,
 5.139972464433231,
 58.00826067003214,
 65.23634694814136,
 5.139972464433231,
 65.23634694814136,
 15.557595227168422,
 5.139972464433231,
 15.557595227168422,
 15.557595227168422,
 5.139972464433231,
 15.557595227168422,
 76.82423129876089,
 18.597980725103263,
 18.597980725103263,
 18.597980725103263,
 24.736117485084893,
 66.72785681505279,
 52.96007342817807,
 19.63056447911886,
 71.77604405690684,
 18.597980725103263,
 18.597980725103263,
 25.53923818265259,
 18.597980725103263,
 18.597980725103263,
 29.55484167049106,
 24.736117485084893,
 72.23497016980268,
 90.59201468563562

# Stress Score when there's clashes